In [ ]:
from google.colab import drive
import sys

drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Research/FairyTaleQA_2')
%cd /content/drive/My\ Drive/Research/FairyTaleQA_2/FairytaleQAData/

Mounted at /content/drive
/content/drive/My Drive/Research/FairyTaleQA_2/FairytaleQAData


In [ ]:
# Uncomment to download original FairytaleQA Dataset
# !git clone https://github.com/uci-soe/FairytaleQAData.git

change the directory to the folder with separate story data

In [ ]:

dir = "./data-by-train-split"
split_version = 'train'

# dir = "./split_for_training/test"
# split_version = 'test'

# dir = "./split_for_training/val"
# split_version = 'val'


In [ ]:
import pandas as pd
import os
import glob
from pathlib import Path
from math import isnan
import csv
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
!ls

data-by-origin	data-by-train-split  img  LICENSE  README.md  starter.py  story_meta.csv


In [ ]:
path = "./data-by-train-split/section-stories/"

In [ ]:
name_set_train = []
name_set_val = []
name_set_test = []

train_cnt_files = 0
val_cnt_files = 0
test_cnt_files = 0


for directory in os.listdir(path):
  folder = os.path.join(path, directory)
  split_value = directory
  for filename in os.listdir(folder):
    name = filename.replace('-story.csv', '')
    if split_value == 'train' and name not in name_set_train:
      name_set_train.append(name)
      train_cnt_files += 1
    elif split_value == 'val' and name not in name_set_val:
      name_set_val.append(name)
      val_cnt_files += 1
    elif split_value == 'test' and name not in name_set_test:
      name_set_test.append(name)
      test_cnt_files += 1


name_set = {'train': name_set_train, 'val': name_set_val, 'test': name_set_test}

assert len(name_set_train) == train_cnt_files
assert len(name_set_val) == val_cnt_files
assert len(name_set_test) == test_cnt_files

print('# of books in train: ', len(name_set_train))
print('# of books in val: ', len(name_set_val))
print('# of books in test: ', len(name_set_test))


print('# of books: ', (len(name_set_train) + len(name_set_val) + len(name_set_test)))


# of books in train:  232
# of books in val:  23
# of books in test:  23
# of books:  278


In [ ]:
def is_nan(x):
    return isinstance(x, float) and isnan(x)

In [ ]:
story_path = "./data-by-train-split/section-stories"
question_path = "./data-by-train-split/questions"

total_data = {}
all_story_data = {}
all_question_data = {}

for split in name_set:
  for story_name in name_set[split]:

    # creates keys in the dictionaries for each story
    if story_name not in total_data:
      total_data[story_name] = []
    if story_name not in all_story_data:
      all_story_data[story_name] = []
    if story_name not in all_question_data:
      all_question_data[story_name] = []

    # reads the csv files for each story and question set and returns them as a dictionary
    story_file_name = '{}/{}/{}-story.csv'.format(story_path, split, story_name)
    question_file_name = '{}/{}/{}-questions.csv'.format(question_path, split, story_name)
    story_data = pd.read_csv(story_file_name).to_dict('index')
    question_data = pd.read_csv(question_file_name).to_dict('index')

    story_data_dict = {}

    full_story = ""

    # processing the each section of a story and concatenating the sentences to create the full story, seperating each section by '<SEP>'
    for section_index in story_data:

      if is_nan(story_data[section_index]['text']):
        continue

      try:
        temp_data = story_data[section_index]['text'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower()
        temp_data = ' '.join(temp_data.split())

        doc = nlp(temp_data)
        tokenized = [token.text for token in doc]
        temp_data = ' '.join(tokenized)

        story_data_dict[int(story_data[section_index]['section'])] = temp_data.replace("n't", 'not')
        all_story_data[story_name].append(temp_data)
        full_story += temp_data + ' <SEP> '

      except:
        print("---", int(story_data[section_index]['section']), story_data[section_index] )

    # REMOVE THE LAST <SEP>
    full_story = full_story[:-6]


    # gathering all the questions and related metadata
    for quest_index in question_data:

      text_section_list = str(question_data[quest_index]['cor_section']).split(',')
      total_text = ''

      if len(text_section_list) == 1:

        question_data[quest_index]['text'] = story_data_dict[int(question_data[quest_index]['cor_section'])]
        total_text = story_data_dict[int(question_data[quest_index]['cor_section'])]
      else:

        for j in text_section_list:
            total_text += story_data_dict[int(j)].strip() + ' '

        question_data[quest_index]['text'] = total_text

      question = question_data[quest_index]['question'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower().strip(' ?') + ' ?'

      # train only has one answer for each question
      if split == 'train':
        answer1 = question_data[quest_index]['answer1'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower().strip(' .') + ' .'

        doc = nlp(answer1)
        tokenized = [token.text for token in doc]
        answer1 = ' '.join(tokenized)


        all_question_data[story_name].append( total_text + ' </s> ' + question + ' </s> ' + answer1 )

      # val and test has two answers for each question
      else:

        answer1 = question_data[quest_index]['answer1'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower().strip(' .') + ' .'
        answer2 = question_data[quest_index]['answer4'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower().strip(' .') + ' .'


        doc = nlp(answer1)
        tokenized = [token.text for token in doc]
        answer1 = ' '.join(tokenized)

        doc = nlp(answer2)
        tokenized = [token.text for token in doc]
        answer2 = ' '.join(tokenized)


        all_question_data[story_name].append( total_text + ' </s> ' + question + ' </s> ' + answer1 + ' </s> ' + answer2 )


      # collecting all the data
      new_data_i = {}
      for k in question_data[quest_index].keys():
          if not is_nan(question_data[quest_index][k]):
              new_data_i[k] = question_data[quest_index][k]

      total_data[story_name].append(new_data_i)








In [ ]:
total_data['Snow-man'][0]

{'question_id': 1,
 'local-or-sum': 'local',
 'cor_section': '1',
 'attribute1': 'action',
 'attribute2': 'setting',
 'question': 'How did the Snow Man describe the weather?',
 'ex-or-im1': 'explicit',
 'answer1': 'delightfully cold',
 'text': '" it is so delightfully cold , " said the snow man , " that it makes my whole body crackle . this is just the kind of wind to blow life into one . how that great red thing up there is staring at me ! " he meant the sun , who was just setting . " it shall not make me wink . i shall manage to keep the pieces . " he had two triangular pieces of tile in his head , instead of eyes . his mouth was made of an old broken rake , and was , of course , furnished with teeth . he had been brought into existence amidst the joyous shouts of boys , the jingling of sleigh - bells , and the slashing of whips . the sun went down , and the full moon rose , large , round , and clear , shining in the deep blue .'}

In [ ]:
!pwd

/content/drive/MyDrive/Research/FairyTaleQA_2/FairytaleQAData


In [ ]:
%cd ../

/content/drive/MyDrive/Research/FairyTaleQA_2


In [ ]:
!rm -r ./fairytaleqa_preprocessed/

rm: cannot remove './fairytaleqa_preprocessed/': No such file or directory


In [ ]:
!mkdir ./fairytaleqa_preprocessed

In [ ]:
%cd ./fairytaleqa_preprocessed/

/content/drive/MyDrive/Research/FairyTaleQA_2/fairytaleqa_preprocessed


In [ ]:
import csv


for split in name_set:
  temp_list = []
  for story_name in name_set[split]:
    _temp = total_data[story_name]
    for question in _temp:
      question['story_name'] = story_name
      temp_list.append(question)
  df = pd.DataFrame(temp_list)
  df.to_csv('fairytaleqa_preprocessed_{}.csv'.format(split), index=False)



In [ ]:
temp_list = []
for story_name in name_set['test']:
  _temp = total_data[story_name]
  for question in _temp:
    question['story_name'] = story_name
    temp_list.append(question)
df = pd.DataFrame(temp_list)
df

,question_id,local-or-sum,cor_section,attribute1,question,ex-or-im1,answer1,ex-or-im2,answer4,text,story_name,answer5,answer2,answer6,attribute2,answer3
0,1,local,1,character,What kind of hair did the wife have?,explicit,golden hair,explicit,golden hair,there was once upon a time a king who had a wi...,alleleiraugh-or-the-many-furred-creature,NaN,NaN,NaN,NaN,NaN
1,2,local,1,causal relationship,Why couldn't one find anyone like the king's w...,explicit,She was so beautiful.,explicit,she was too beautiful,there was once upon a time a king who had a wi...,alleleiraugh-or-the-many-furred-creature,no one had golden hair,NaN,NaN,NaN,NaN
2,3,local,1,outcome resolution,What happened when the king's wife fell ill?,explicit,She felt that she must soon die.,explicit,she felt she must soon die,there was once upon a time a king who had a wi...,alleleiraugh-or-the-many-furred-creature,NaN,NaN,NaN,NaN,NaN
3,4,local,1,character,Who did the king's wife send for when she felt...,explicit,the king,explicit,the king,there was once upon a time a king who had a wi...,alleleiraugh-or-the-many-furred-creature,NaN,NaN,NaN,NaN,NaN
4,5,local,1,causal relationship,Why did the councillors say the king had to ma...,explicit,so that they may have a queen,explicit,so that we may have a queen,there was once upon a time a king who had a wi...,alleleiraugh-or-the-many-furred-creature,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,52,local,12,causal relationship,What did the widowed Mistress offer after the ...,explicit,herself,explicit,Take herself,""" oh , sweet madam mistress , "" she cried , "" ...",whippety-stourie,NaN,NaN,NaN,NaN,NaN
1003,53,local,12,feeling,How did the Old Woman feel after the widowed M...,implicit,astonished,explicit,Astonishment,""" oh , sweet madam mistress , "" she cried , "" ...",whippety-stourie,NaN,surprised,NaN,NaN,amazed
1004,54,local,12,causal relationship,Why didn't the Old Woman want to take the wido...,implicit,The widowed Mistress was not pretty.,explicit,"She was a plain-looking, red-eyed, dowdy wife",""" oh , sweet madam mistress , "" she cried , "" ...",whippety-stourie,NaN,The widowed Mistress had no beauty.,NaN,NaN,NaN
1005,55,summary,"12, 13",causal relationship,Why didn't the widowed Mistress tease the Old ...,implicit,The widowed Mistress became angry at her.,implicit,She got mad at the Old woman,""" oh , sweet madam mistress , "" she cried , "" ...",whippety-stourie,The Old woman's tone made her angry,The widowed Mistress got mad.,NaN,NaN,NaN


In [ ]:
!ls

fairytaleqa_preprocessed_test.csv   fairytaleqa_preprocessed_val.csv
fairytaleqa_preprocessed_train.csv
